In [45]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from xgboost import XGBClassifier, plot_importance
import lightgbm as lgb
from lightgbm import LGBMClassifier, plot_importance, plot_tree
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler

import warnings
from tqdm import tqdm

import itertools
warnings.filterwarnings('ignore')

# 데이터 불러오기

In [46]:
train=pd.read_csv('data/train.csv')
test=pd.read_csv('data/test.csv')

# 파생변수 생성 및 인코딩

In [47]:
## 나이 양수로 변환
train['birth_plus'] = train.DAYS_BIRTH*(-1)/365
test['birth_plus'] = test.DAYS_BIRTH*(-1)/365

train['gender']=LabelEncoder().fit_transform(train.gender)
train['car']=LabelEncoder().fit_transform(train.car)
train['reality']=LabelEncoder().fit_transform(train.reality)
train['income_type']=LabelEncoder().fit_transform(train.income_type)
train['edu_type']=LabelEncoder().fit_transform(train.edu_type)
train['family_type']=LabelEncoder().fit_transform(train.family_type)
train['house_type']=LabelEncoder().fit_transform(train.house_type)
train.fillna('NAN',inplace=True)
train['occyp_type']=LabelEncoder().fit_transform(train.occyp_type)
train['income_weight']=train.income_total**2
train['income_age']=train.income_total*train.DAYS_BIRTH
train['income_emp']=train.income_total*train.DAYS_EMPLOYED

test['gender']=LabelEncoder().fit_transform(test.gender)
test['car']=LabelEncoder().fit_transform(test.car)
test['reality']=LabelEncoder().fit_transform(test.reality)
test['income_type']=LabelEncoder().fit_transform(test.income_type)
test['edu_type']=LabelEncoder().fit_transform(test.edu_type)
test['family_type']=LabelEncoder().fit_transform(test.family_type)
test['house_type']=LabelEncoder().fit_transform(test.house_type)
test.fillna('NAN',inplace=True)
test['occyp_type']=LabelEncoder().fit_transform(test.occyp_type)
test['income_weight']=test.income_total**2
test['income_age']=test.income_total*test.DAYS_BIRTH
test['income_emp']=test.income_total*test.DAYS_EMPLOYED

train.income_total=train.income_total.astype(int)
train.family_size=train.family_size.astype(int)
train.begin_month=train.begin_month.astype(int)
train.birth_plus=train.birth_plus.astype(int)

test.income_total=test.income_total.astype(int)
test.family_size=test.family_size.astype(int)
test.begin_month=test.begin_month.astype(int)
test.birth_plus=test.birth_plus.astype(int)

train.credit=train.credit.astype('category')
train.drop(['index','family_size','work_phone'],axis=1,inplace=True)
test.drop(['index','family_size','work_phone'],axis=1,inplace=True)
train.DAYS_BIRTH=train.DAYS_BIRTH**6
test.DAYS_BIRTH=test.DAYS_BIRTH**6
train.income_type=1/(train.income_type)
test.income_type=1/(test.income_type)
train.income_weight=1/(train.income_weight)**3
test.income_weight=1/(test.income_weight)**3
train['DAYS_BIRTH_weight']=1/(train.DAYS_BIRTH)**2
test['DAYS_BIRTH_weight']=1/(test.DAYS_BIRTH)**2

In [52]:
## predict_rf
ftr = train.drop('credit', axis=1).values
target = train['credit'].values
tst_ar = test.values
n_class = 3
n_fold = 17

In [61]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=40)

print(ftr.shape)

lgb_p_val = np.zeros((ftr.shape[0], n_class))
lgb_p_tst = np.zeros((tst_ar.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr, target), 1):
    print(f'training model for CV #{i}')
    lgb_clf = lgb.LGBMClassifier(max_depth=24,
                       num_leaves=110,
                       colsample_bytree=0.3,
                       n_estimators=175, 
                       min_child_samples=2,
                       subsample=0.9,
                       subsample_freq=2,
                       learning_rate=0.09,
                       random_state=2021,
                       verbose = 50)

    lgb_clf.fit(ftr[i_trn], target[i_trn],
            eval_set=[(ftr[i_val], target[i_val])],
            eval_metric='multi_error')

    lgb_p_val[i_val, :] = lgb_clf.predict_proba(ftr[i_val])
    lgb_p_tst += lgb_clf.predict_proba(tst_ar) / n_fold
    
print(f'{log_loss(target, lgb_p_val)}')
print(f'{confusion_matrix(target, np.argmax(lgb_p_val, axis=1))}%')

(26457, 21)
training model for CV #1
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.748574
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.247245
[LightGBM] [Debug] init for col-wise cost 0.001042 seconds, init for row-wise cost 0.001309 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001246 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1758
[LightGBM] [Info] Number of data points in the train set: 24900, number of used features: 20
[LightGBM] [Debug] Use subset for bagging
[LightGBM] [Info] Start training from score -2.105645
[LightGBM] [Info] Start training from score -1.440254
[LightGBM] [Info] Start training from score -0.444156
[LightGBM] [Debug] Re-bagging, using 22513 data to train
[LightGBM] [Debug] Trained a tree with leave

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 20
[34]	valid_0's multi_error: 0.298009	valid_0's multi_logloss: 0.75266
[LightGBM] [Debug] Re-bagging, using 22503 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[35]	valid_0's multi_error: 0.291586	valid_0's multi_logloss: 0.749459
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 20
[36]	valid_0's multi_error: 0.289017	valid_0's multi_logloss: 0.748814
[LightGBM] [Debug] Re-bagging, using 22414 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[37]

[72]	valid_0's multi_error: 0.26718	valid_0's multi_logloss: 0.708622
[LightGBM] [Debug] Re-bagging, using 22386 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 12
[73]	valid_0's multi_error: 0.269107	valid_0's multi_logloss: 0.708013
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[74]	valid_0's multi_error: 0.268465	valid_0's multi_logloss: 0.707185
[LightGBM] [Debug] Re-bagging, using 22368 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[75]	valid_0's multi_error: 0.26975	valid_0's multi_logloss: 0.706569
[

[LightGBM] [Debug] Re-bagging, using 22418 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 20
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[111]	valid_0's multi_error: 0.265254	valid_0's multi_logloss: 0.689814
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[112]	valid_0's multi_error: 0.266538	valid_0's multi_logloss: 0.68894
[LightGBM] [Debug] Re-bagging, using 22423 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[113]	valid_0's multi_error: 0.26718	valid_0's multi_logloss: 0.688566
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[L

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[146]	valid_0's multi_error: 0.268465	valid_0's multi_logloss: 0.682046
[LightGBM] [Debug] Re-bagging, using 22353 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[147]	valid_0's multi_error: 0.269107	valid_0's multi_logloss: 0.68203
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 22
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[148]	valid_0's multi_error: 0.267823	valid_0's multi_logloss: 0.681358
[LightGBM] [Debug] Re-bagging, using 22291 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[

...

[25]	valid_0's multi_error: 0.327121	valid_0's multi_logloss: 0.782027
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[26]	valid_0's multi_error: 0.316838	valid_0's multi_logloss: 0.779203
[LightGBM] [Debug] Re-bagging, using 22388 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[27]	valid_0's multi_error: 0.312982	valid_0's multi_logloss: 0.777033
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 20
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[28]	valid_0's multi_error: 0.309769	valid_0's multi_logloss: 0.772967
[LightGBM] [Debug] Re-bagging, using 22443 data to train

[LightGBM] [Debug] Re-bagging, using 22432 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[63]	valid_0's multi_error: 0.290488	valid_0's multi_logloss: 0.726565
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 20
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[64]	valid_0's multi_error: 0.291774	valid_0's multi_logloss: 0.726045
[LightGBM] [Debug] Re-bagging, using 22428 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[65]	valid_0's multi_error: 0.289846	valid_0's multi_logloss: 0.725395
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[Li

[93]	valid_0's multi_error: 0.280206	valid_0's multi_logloss: 0.711995
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[94]	valid_0's multi_error: 0.280206	valid_0's multi_logloss: 0.711604
[LightGBM] [Debug] Re-bagging, using 22478 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[95]	valid_0's multi_error: 0.278278	valid_0's multi_logloss: 0.71155
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 22
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[96]	valid_0's multi_error: 0.27635	valid_0's multi_logloss: 0.711244
[LightGBM] [Debug] Re-bagging, using 22392 data to train
[

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[121]	valid_0's multi_error: 0.277635	valid_0's multi_logloss: 0.704013
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[122]	valid_0's multi_error: 0.275707	valid_0's multi_logloss: 0.703535
[LightGBM] [Debug] Re-bagging, using 22413 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[123]	valid_0's multi_error: 0.276992	valid_0's multi_logloss: 0.703396
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and d

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[150]	valid_0's multi_error: 0.271851	valid_0's multi_logloss: 0.702493
[LightGBM] [Debug] Re-bagging, using 22397 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[151]	valid_0's multi_error: 0.270566	valid_0's multi_logloss: 0.702064
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 13
[152]	valid_0's multi_error: 0.272494	valid_0's multi_logloss: 0.702837
[LightGBM] [Debug] Re-bagging, using 22384 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17


In [62]:
print(f'{log_loss(target, lgb_p_val)}')
print(f'{confusion_matrix(target, np.argmax(lgb_p_val, axis=1))}%')

0.6811907370591721
[[  897   435  1890]
 [  221  2867  3179]
 [  414   782 15772]]%


----

In [66]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'lgb'
feature_name = '2021'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, lgb_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, lgb_p_tst, fmt='%.6f', delimiter=',')

In [ ]:
sample_submission=pd.read_csv('data/sample_submission.csv')
for i in range(17):
    sample_submission.iloc[:,1:]+=models_lgbm[i].predict_proba(test)/17
    
sample_submission.to_csv('state2021.csv',index=False)

In [29]:
feature_lgbm=feature_lgbm.sort_index()
feature_lgbm

print(np.mean(np.array(logloss_avg)))

0.6880214218100895


In [30]:
print(np.mean(np.array(logloss_avg)))

0.6880214218100895


- 90 : 0.6890796591713713

----

In [28]:
############################################################################
label=train.credit
train0=train.drop('credit',axis=1)
skf=StratifiedKFold(n_splits=17,random_state=40,shuffle=True)
folds=[]
for train_idx,val_idx in skf.split(train0,label):
    folds.append([train_idx,val_idx])
models_lgbm={}
logloss_avg=[]
feature_lgbm=pd.DataFrame()
for i in range(17):
    train_x,val_x,train_y,val_y=train0.iloc[folds[i][0],:],train0.iloc[folds[i][1],:],label[folds[i][0]],label[folds[i][1]]
    lgb=LGBMClassifier(max_depth=24,
                       num_leaves=110,
                       colsample_bytree=0.3,
                       n_estimators=230, 
                       min_child_samples=2,
                       subsample=0.9,
                       subsample_freq=2,
                       learning_rate=0.09,
                       random_state=99,
                       verbose = 50
                       # reg_alpha=0.1
                    )
    lgb.fit(train_x,train_y)
    logloss_avg.append(log_loss(val_y,lgb.predict_proba(val_x)))
    models_lgbm[i]=lgb
    feature_lgbm=pd.concat([feature_lgbm,pd.DataFrame(lgb.predict_proba(val_x),index=folds[i][1])],axis=0)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.748574
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.247245
[LightGBM] [Debug] init for col-wise cost 0.001282 seconds, init for row-wise cost 0.001238 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001496 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1758
[LightGBM] [Info] Number of data points in the train set: 24900, number of used features: 20
[LightGBM] [Debug] Use subset for bagging
[LightGBM] [Info] Start training from score -2.105645
[LightGBM] [Info] Start training from score -1.440254
[LightGBM] [Info] Start training from score -0.444156
[LightGBM] [Debug] Re-bagging, using 22369 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [De

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Re-bagging, using 22402 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Re-bagging, using 22422 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Deb

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22462 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Re-bagging, using 22376 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 20
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Deb

[LightGBM] [Debug] Re-bagging, using 22430 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Re-bagging, using 22393 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22406 data to train
[LightGBM] [Debug] Traine

[LightGBM] [Debug] Re-bagging, using 22414 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Re-bagging, using 22472 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Re-bagging, using 22369 data to train
[LightGBM] [Debug] Traine

...

[LightGBM] [Debug] Re-bagging, using 22451 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Re-bagging, using 22398 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22408 data to train
[LightGBM] [Debug] Traine

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22398 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22358 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Deb

[LightGBM] [Debug] Re-bagging, using 22420 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Re-bagging, using 22358 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22459 data to train
[LightGBM] [Debug] Traine

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 22
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22385 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Re-bagging, using 22443 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debu

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Re-bagging, using 22420 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 22
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22428 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Deb